In [148]:
from ipynb.fs.full.feature_engineering_final import X_Transformer_scaled
from ipynb.fs.full.feature_engineering_final import X_Transformer
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_squared_log_error
from ipynb.fs.full.feature_engineering_final import DataLoader
from sklearn.linear_model import BayesianRidge
from ipynb.fs.full.feature_engineering_final import TargetNormalizedRegressor
from sklearn.kernel_ridge import KernelRidge
from lightgbm import LGBMRegressor
from ipynb.fs.full.feature_engineering_final import BlendingRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV
import xgboost as xgb
import numpy as np
import pandas as pd
from xgboost import plot_tree
from mlxtend.regressor import StackingCVRegressor
from mlxtend.regressor import StackingRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import plot_tree
from matplotlib.pylab import rcParams
from sklearn.model_selection import KFold
from sklearn.svm import SVR

In [149]:
data_loader = DataLoader("train_r.csv","test_r.csv")

/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [150]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()
X_test = data_loader.getX_test()

In [151]:
x_transformer = X_Transformer_scaled()

In [152]:
kf = KFold(n_splits=10,shuffle = True,random_state = 42)

In [153]:
en = ElasticNetCV(max_iter=1e7, alphas=[0.0003,0.0004,0.00045,0.0005,0.0006], cv=kf, 
                  l1_ratio=[0.5,0.6,0.7,0.8,0.9])
en1 = ElasticNetCV(cv = kf)
kr =  KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=9.035)
huber = HuberRegressor(alpha = 265,epsilon=1.6)
ridge = RidgeCV(alphas=[14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5,15.5,15.6])
bayesian_ridge = BayesianRidge()
lasso = LassoCV(cv=10)
svr = SVR(kernel="rbf",gamma=0.0003,C = 14,epsilon = 0.00008)
gbr = GradientBoostingRegressor(n_estimators=5950, learning_rate=0.01,
                                   max_depth=4, max_features='sqrt',
                                   loss='huber', random_state =42,subsample = 0.7)
xgb = XGBRegressor(objective='reg:squarederror',eval_metric = 'rmse',learning_rate=0.01,n_estimators=4900,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

scvr = StackingCVRegressor(regressors=(en,ridge,lasso,gbr,svr,xgb),
                                meta_regressor=en,cv=15,shuffle = True,random_state = 42,
                                use_features_in_secondary=True,
                                  n_jobs = -1)

scvr_t = TargetNormalizedRegressor(scvr)


br1 = BlendingRegressor(regressors_and_weights = [(en,0.65),(kr,0.35)])
br = BlendingRegressor(regressors_and_weights = [(br1,0.78),(huber,0.22)])
br_t = TargetNormalizedRegressor(br)

In [154]:
learning_pipeline = Pipeline(steps=[('preprocess', x_transformer),('learn',scvr_t)])
learning_pipeline.fit(X_train,y=Y_train)





Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=None,
                          steps=[('col_drop',
                                  ColumnDrop(columns=['Utilities', 'Street',
                                                      'PoolQC'])),
                                 ('col_transformer',
                                  ColumnTransformer(n_jobs=None,
                                                    remainder='passthrough',
                                                    sparse_threshold=0.3,
                                                    transformer_weights=None,
                                                    transformers=[('categorical_with_missing_values_const',
                                                                   PipelineWithFeatureNames(feature_names=['...
                                                                                                  learning_rate=0.01,
                                 

In [155]:
#set up the parameters
#rcParams['figure.figsize'] = 100,90
#plot_tree(learning_pipeline.named_steps['learn'],num_trees=2799, rankdir='LR')#plt.show()


In [156]:
Ypredicted = learning_pipeline.predict(X_test)

In [157]:
resultDf = pd.DataFrame()
resultDf['Id'] = data_loader.get_Test_id()
resultDf['SalePrice'] = Ypredicted

#q1 = resultDf['SalePrice'].quantile(0.0045)
#q2 = resultDf['SalePrice'].quantile(0.99)
#resultDf[resultDf['SalePrice'] <= q1]
#resultDf[resultDf['SalePrice'] <= q1]

#resultDf['SalePrice'] = resultDf['SalePrice'].apply(lambda x: x if x > q1 else x*0.9)
#resultDf['SalePrice'] = resultDf['SalePrice'].apply(lambda x: x if x < q2 else x*1.081)

#resultDf
resultDf.to_csv('submission.csv',index = False)